In [1]:
import pandas as pd
import numpy as np
import nltk
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read for backend work
df = pd.read_csv(r"C:\Users\dsk02\Desktop\python_projects\nn_recipes\cleaned_data.csv")
df = df.drop(columns=['Unnamed: 0'])

In [3]:
df.head()

,Recipe,Ingredients
0,pan fried chicken goujons,"flour,cocoa,powder,ground,pepper,chicken,fille..."
1,vegetable soup with couscous,"green,pepper,medium,tomato,onion,carrot,potato..."
2,cheesy leek and mustard soup,"leek,onion,medium,potato,carrot,butter,olive,o..."
3,goalkeeper s steak and ale pie,"butter,steak,carrot,swede,potato,onion,bitter,..."
4,christmas stollen,"active,yeast,milk,egg,sugar,salt,butter,bread,..."


###### some ingredients are "ground" etc. find a way to remove things that aren't real ingredient or relevant to api
###### use ntlk to tag words, only keeping specific tagged words

In [19]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize as WordTokenizer
lemma = WordNetLemmatizer()

def ntlk_edits(df):
    
    # temp array
    temp = []
    
    # split each word in the dataframe at the ,
    split_words = (df['Ingredients'].split(','))
    
    for each_word in split_words:
        
        # lowercase each word
        each_word = each_word.lower()
        
        # not a space or punctuation
        if each_word.isalpha():
            
            #print('this is before lemma {}'.format(each_word))
            
            # tokenize each word, sometimes pushes to array
            #words = WordTokenizer(each_word)
            
            # check to see if the words got pushed to an array
            #if len(words) > 1:
                
            # ipush to array handle with for
            #for word in words:

            # lemmatize each word for standard processing
            lemm_word = lemma.lemmatize(each_word)
            
            #print('this is after lemma {}'.format(lemm_word))

            # tag each word based on nltks position tagger
            check = nltk.pos_tag([lemm_word]) # have to pass as arr -> tuple
            
            #print('this is after pos_tag {}'.format(check))

            # append 'NN' (nouns) words to the temp array after check
            if check[0][1] == 'NN': #nested array
                temp.append(check[0][0]) #nested array
                #print(check[0][1]) check nn       
                
    return temp

In [20]:
df['nltk_tagging'] = df.apply(ntlk_edits, axis=1)

In [23]:
# have cleaned out some of the more uesless words
df.loc[1337,['Ingredients','nltk_tagging']]

Ingredients       flour,egg,
nltk_tagging    [flour, egg]
Name: 1337, dtype: object

In [24]:
df

,Recipe,Ingredients,nltk_tagging
0,pan fried chicken goujons,"flour,cocoa,powder,ground,pepper,chicken,fille...","[flour, cocoa, powder, ground, pepper, chicken..."
1,vegetable soup with couscous,"green,pepper,medium,tomato,onion,carrot,potato...","[pepper, medium, tomato, onion, carrot, potato..."
2,cheesy leek and mustard soup,"leek,onion,medium,potato,carrot,butter,olive,o...","[leek, onion, medium, potato, carrot, butter, ..."
3,goalkeeper s steak and ale pie,"butter,steak,carrot,swede,potato,onion,bitter,...","[butter, steak, carrot, swede, potato, onion, ..."
4,christmas stollen,"active,yeast,milk,egg,sugar,salt,butter,bread,...","[yeast, milk, egg, sugar, salt, butter, bread,..."
...,...,...,...
53856,mediterranean chicken tray bake,"pepper,cut,onion,wedge,olive,oil,chicken,garli...","[pepper, cut, onion, wedge, oil, chicken, garl..."
53857,honeycomb,"butter,cream,make,sugar,syrup,a,soda,bread,","[butter, cream, sugar, syrup, soda, bread]"
53858,crab and leek pasties,"puff,pastry,butter,cream,make,leek,garlic,onio...","[puff, pastry, butter, cream, leek, garlic, on..."
53859,horseradish potato dauphinoise,"cream,milk,ingredient,a,garlic,clove,horseradi...","[cream, milk, ingredient, garlic, clove, horse..."


##### build dictionary for main backbone, serialize afterwards

In [92]:
def dictionary_create(recipes, ingredients):
    
    # create dictionary
    recipe_dict = {}
    
    print(len(ingredients),len(recipes))
    
    # ingredients is an array of arrays
    for recipe, arr in zip(recipes, ingredients):

        # access each value in each array
        for each_ingredient in arr: # N^2 :(
            
            # check if it's in the dictinoary
            if each_ingredient not in recipe_dict:
                
                # if not add a array to hold values
                recipe_dict[each_ingredient] = []
                # # at each ingredient, push the recipe name into the array
                recipe_dict[each_ingredient].append(recipe)
            else:  
                # at each ingredient, push the recipe name into the array
                recipe_dict[each_ingredient].append(recipe)

    return recipe_dict


In [94]:
# convert series to arrays from df
recipes = list(df['Recipe'])
ingredients = list(df['nltk_tagging'])

my_recipes = dictionary_create(recipes,ingredients)

print(len(my_recipes))


53861 53861
1092


In [96]:
# for example, all the recipes we could make with chicken (first 5)
my_recipes['chicken'][0:5]

['pan fried chicken goujons',
 'garlic chicken in puff pastry',
 'indian masala chicken drumsticks',
 'easy quesadillas',
 'apple sauce pork chops']

In [97]:
# our suggestions will be in the intersections of the arrays
# (or intersection of a bunch)
# for example
# my_recipes['chicken'] ∩ my_recipes['fries'] = chicken and fries!

In [98]:
# save my dictionary

import pickle

with open(r"C:\Users\dsk02\Desktop\python_projects\nn_recipes\recipes_dict.pickle", 'wb') as handle:
    pickle.dump(my_recipes, handle, protocol=pickle.HIGHEST_PROTOCOL)

# opening
#with open(r"C:\Users\dsk02\Desktop\python_projects\nn_recipes\recipes_dict.pickle", 'rb') as handle:
#    b = pickle.load(handle)

#print a == b